## Integrantes:
1. Camila Coltriani
2. Luis Dartayet
3. Irania Fuentes
4. Jonathan Fichelson
5. Ornella Cevoli

# Trabajo práctico 2 : Modelo de regresión lineal del dataset Properatti

## Objetivos

El objetivo de este trabajo final es generar y comparar estadísticamente tres modelos de regresión lineal sobre el dataset limpio de Properatti construido en el TP_1; en este, fue planteado la hipótesis que el precio (variable objetivo) de las propiedades iba a estar influenciado principalmente por la superficie y la ubicación (variables predictoras). 

Con base a esto, se han planteado los siguientes objetivos específicos:
- Explorar el dataset limpio con la finalidad de verificar si debe realizarse una ultima limpieza o pueden utilizase los datos directamente;
- Realizar una visualización general de las distribuciones y relaciones del dataset con la finalidad de determinar la zona, tipo de inmueble y variables predictoras y objetivo para la realización de los modelos;
- Construir modelos de regresión lineal simple y multiple e interpretar sus metricas con la finalidad de identificar el que mejor permita obtener una predicción confiable de la variable objetivo;
- Implementar un modelo de regularización con la finalidad de compararlos y evaluar si existe o no problemas de sobreajuste;
- Determinar el modelo que más se ajusta al comportamiento de los datos analizados. 

In [ ]:
#Las librerías utilizadas en este documento son:
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from sklearn import metrics
from matplotlib.ticker import ScalarFormatter
from matplotlib import gridspec
sns.set()

## Visualización del dataset

In [ ]:
# Visualización de la forma y atributos del dataset
data = pd.read_csv("./data/data_limpio_gdf.csv")
print(data.shape)
print("El dataset está compuesto por:", data.shape[0], "filas y",data.shape[1],"columnas.")
data.sample(5)

### Descripción de las columnas del dataset limpio:

Las columnas que incluye son:

● municipio: ubicación del inmueble por su municipio/barrios

● provincia: ubicación del inmueble por provincia

● lat  ●lon: ubicación de latitud y longitud

● superficie_m2_total: superficie total en m² del inmueble

● price_usd: Precio en dólares del inmueble

● tipo: tipo de inmueble en venta (casa, departamento, ph, tienda)

● ambientes_cat: cantidad de ambientes del inmueble (0, 1, 2, 3 , 4 o más)

● precio_usd_por_m2: Precio en dólares por metro cuadrado (USD/m²: precio dólares / superficie)

● tipo_cat_code: categoría numérica de tipo de inmueble

● municipio_cat_code: categoría numérica de municipios

● provincia_cat_code: categoría numérica de provincia

● tipo_cat_code: categoría numérica de ambientes_cat

● geometry: figura geométrica de latitud y la longitud

● country_name: nombre del país donde ocurre la operación inmobiliaria

● **precio_usd_por_m2_cat: categoría numérica de precio_usd_por_m2**

# Análisis exploratorio y visualización de correlaciones entre las variables

In [ ]:
#Revisamos la presencia de datos NaN
data.isna().sum().sort_values()
#La columna "ambientes_cat" quedó con 1248 registros nulos

In [ ]:
#reviso donde están ubicados y a que propiedad pertenecen los registros nulos para saber si afectaran escoger un tipo de inmueble y su zona
mascara_nulos = data["ambientes_cat"].astype(str) == "nan" 
data_nulos = data[mascara_nulos]
data_nulos.loc[:, ["municipio", 'tipo', 'ambientes_cat', "precio_usd"]].sample(7)
#print(data[mascara_nulos].index)

In [ ]:
#agrupamos los registros donde hay nulos
pd.options.display.max_rows = None
data_nulos.groupby(["tipo"])["municipio"].value_counts().sort_values(ascending=False)
#vemos que los nan están distribuidos equitativamente y no están concentrados en una mismo municipio

In [ ]:
#Los elimino p
data.dropna(subset=['ambientes_cat'], inplace=True)
print(data.isna().sum())

In [ ]:
# Eliminamos algunos outliers

data= data[(data["precio_usd"] >100) & (data["precio_usd"]<= 4.500000e+05)]
data= data[(data["precio_usd_por_m2"] > 100) & (data["precio_usd_por_m2"]<= 2.500000e+04)]

In [ ]:
#Realizamos una descripción estadística del dataset
data.describe()
#existen datos que no permiten ver los estadísticos ya que hay valores de 0 en sup_m2_total e inf en precio_usd_por_m2: eliminarlos

In [ ]:
#eliminamos del dataset los registros de sup_m2_total con valores de cero
data.drop(data[(data["sup_m2_total"] ==0)].index, inplace=True ,axis=0)

In [ ]:
data.describe()

In [ ]:
#Vemos la correlación entre las variables 
data_corr = data.corr()
#graficamos
plt.figure(figsize=(6,6))
sns.heatmap(data_corr, annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Correlación entre variables")
plt.show()

#puede verse una correlación significativa entre sup_m2_total y precio_usd (0.39)
#tipo_cat_code / ambientes_cat_code y  precio_usd (0.30)
#precio_usd_por_m2_cat y lon

In [ ]:
#graficamos las provincias y municipios que contengan un valor mínimo de 500 registros por municipio (para una mejor visualización)
limite = 500
data = data.copy().groupby(['municipio']).filter(lambda grp: grp.shape[0] > limite)

In [ ]:
fig= plt.subplots(figsize=(20,20),constrained_layout=True)
grid = gridspec.GridSpec(2, 1, height_ratios=[1, 3])

ax1=plt.subplot(grid[0])
sns.countplot(data=data,y="provincia",order=data["provincia"].value_counts().index ,ax=ax1,color="g")

ax1.set_yticklabels(ax1.get_yticklabels(),fontsize="medium")
ax1.set_title("Distribucion de registros segun la provincia", fontsize= 'large')

ax2=plt.subplot(grid[1])
sns.countplot(data=data,x="municipio",order=data["municipio"].value_counts().index,ax=ax2,color="b")


ax2.set_title("Distribucion de registros segun los municipios", fontsize= 'large')
ax2.set_xticklabels(ax2.get_xticklabels(),rotation=90,ha="right")
plt.xticks(fontsize= 10)
plt.yticks(fontsize= 10)
ax1.grid()
ax2.grid()
plt.show()

La mayor cantidad de registros están Capital Federal para los barrios de Palermo, Belgrano, Caballito..
Consideraremos Capital Federal para la evaluación de los modelos 

In [ ]:
#Revisamos la distribución de registros por tipo de inmueble
plt.figure(figsize=(5,3))

plt.gca().yaxis.set_major_formatter(ScalarFormatter())
ax = sns.countplot(data = data, x = "tipo")
ax.set_xticklabels(ax.get_xticklabels(),rotation=40,ha="right")
plt.show()

#Apartamentos tiene la mayoría de los datos

Consideraremos apartamentos para la evaluacion de los modelos

In [ ]:
#Revisamos la ubicacion de los los tipos de inmueble que contienen la mayor cantidad de registros
pd.options.display.max_rows = None
data.groupby(["municipio"])["tipo"].value_counts().sort_values(ascending=False)

#### Para determinar en que barrio enfocarnos, generaremos una funcion que correremos para los tres barrios enunciados 

In [ ]:
# Seleccionamos las columnas que vamos a utilizar y sólo las propiedades de tipo apartamento

df_0 = data[data['tipo'] == 'apartment']
df_0 = data[['municipio', 'sup_m2_total', 'ambientes_cat', 'lat', 'lon', 'precio_usd' ]]
df_0.head()

In [ ]:
# Extraemos un dataframe por cada columna

df_0_sup_m2_total = df_0[['sup_m2_total', 'precio_usd', 'municipio']]
df_0_ambientes_cat = df_0[['ambientes_cat', 'precio_usd', 'municipio']]
df_0_ubica = df_0[['lat', 'lon', 'precio_usd', 'municipio']]

In [ ]:
# Armamos la función para las regresiones lineales

def regresion_lineal(df_0_lr, target, test_size=0.2, random_state=42):
    # Separamos la variable objetivo del resto del dataset
    X = df_0_lr.drop(target, axis=1)
    y = df_0_lr[target]
    
    # Dividimos el dataset en train y test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    # Normalizamos los datos
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Instanciamos el modelo
    model = linear_model.LinearRegression()
    
    # Entrenamos el modelo
    model.fit(X_train, y_train)
    
    # Predecimos con el modelo
    y_pred = model.predict(X_test)
    
    # Calculamos el R2
    r2 = r2_score(y_test, y_pred)
    
    # Imprimimos los resultados
    print('R2: ', r2)
    print('Intercept: ', model.intercept_)
    print('Coeficiente: ', model.coef_[0])
    
    # Graficamos los resultados
    plt.figure(figsize=(4, 3))
    plt.scatter(y_test, y_pred)
    plt.axline(xy1=(0, 0), slope=1, color='red')
    plt.xlabel('Valores reales')
    plt.ylabel('Valores predichos')
    plt.xlim(0, 1000000)
    plt.ylim(0, 1000000)
    plt.show()

In [ ]:
#creamos una mascara para Palermo, Belgrano, Caballito.
municipios = np.array(['Palermo', 'Belgrano', 'Caballito'])

Elegimos estos 3 barrios porque son los que tienen mayor cantidad de registros y mejor R2 inicial en el modelo de regresión lineal simple.
> confrontar con el archivo lr-barrios-capital.ipynb donde se corrió el modelo para todos los barrios de capital federal


In [ ]:
# Regresión lineal para el precio de las propiedades en función del municipio y la superficie total y guardamos el modelo en un mapa

for municipio in municipios:
    df_0_municipio =df_0_sup_m2_total[df_0_sup_m2_total['municipio'] == municipio]
    if(df_0_municipio.shape[0] > 100):
        df_0_municipio = df_0_municipio.drop('municipio', axis=1)
        print('Municipio: ', municipio)
        regresion_lineal(df_0_municipio, 'precio_usd')
        print('----------------------------------------')

## Correlación entre la variables del dataset

In [ ]:
#analizamos la correlación entre cada una de las variables.
figz= plt.figure()
mask_cols= ["tipo","sup_m2_total","precio_usd","precio_usd_por_m2", "ambientes_cat", "municipio_cat_code", "ambientes_cat_code","lat"]
graph=sns.pairplot(data[mask_cols],hue="tipo")
graph.fig.set_size_inches(20,10)
plt.grid()
plt.show()


In [ ]:
# Realizamos los siguientes gráficos para visualizar mejor (zoom) las relaciones- En primera medida analizamos metros totales con precio en dolares
g = sns.FacetGrid(data, col="tipo")
g.map(sns.scatterplot, "sup_m2_total", "precio_usd", alpha=.5)
g.add_legend()


In [ ]:
#

g = sns.FacetGrid(data, col="tipo")
g.map(sns.scatterplot, "sup_m2_total", "precio_usd_por_m2", alpha=.5)
g.add_legend()

In [ ]:
g = sns.FacetGrid(data, col="tipo")
g.map(sns.scatterplot, "precio_usd_por_m2", "precio_usd", alpha=.7)
g.add_legend()

In [ ]:
g = sns.FacetGrid(data, col="tipo")
g.map(sns.scatterplot, "municipio_cat_code", "precio_usd", alpha=.7)
g.add_legend()

In [ ]:
g = sns.FacetGrid(data, col="tipo")
g.map(sns.scatterplot, "ambientes_cat_code", "precio_usd", alpha=.7)
g.add_legend()

#### la variable target debe tener una distribucion cercana a la normal

In [ ]:
#de acuerdo con los valores mínimos y máximos realizamos un histograma para ver la distribución de datos de precio_usd
data_2= data[(data["precio_usd"] >10000) & (data["precio_usd"]<= 4.500000e+05)]
sns.histplot(data_2["precio_usd"], color = "orange", bins = 40)

In [ ]:
#de acuerdo con los valores mínimos y máximos realizamos un histograma para ver la distribución de datos de superficie total
data_2= data[(data["sup_m2_total"] >10) & (data["sup_m2_total"]<= 400)]
sns.histplot(data_2["sup_m2_total"], color = "green", bins = 40)

In [ ]:
data_2= data[(data["precio_usd_por_m2"] >10) & (data["precio_usd_por_m2"]<= 2.812500e+03)]
sns.histplot(data_2["precio_usd_por_m2"], color = "blue", bins = 20)


In [ ]:
data.boxplot(column= "precio_usd", by="tipo") 
plt.show()

Conclusiones del análisis de variables predictoras y target:
- La mayor cantidad de registros están Capital Federal para los barrios de Palermo, Belgrano, Caballito. Seleccionaremos el barrio Caballito porque que tiene una buena cantidad de registros y un buen R2 inicial en el modelo de regresión lineal simple inicial.
- Utilizaremos departamentos como el tipo de inmueble a modelar por contener una mayor cantidad de datos
- Consideraremos como variables predictoras: Superficie total y precio usd por su alta correlación, y su distribución. Luego evaluaremos el impacto de las variables de ubicación y ambientes.
